In [1]:
# import in each cell because of kernel restarts
# better to run in comman line because of restarts
import scrapy
from scrapy.crawler import CrawlerProcess

class ESSpider(scrapy.Spider):
    # Naming the spider is important if running more than one
    # of this class simultaneously
    name = 'ESS'
    
    # URL(s) to start with
    start_urls = [
        'http://www.everydaysexism.com',
    ]
    
    # What to do with the URL.  Download to mainpage.html of file
    def parse(self, response):
        with open('mainpage.html', 'wb') as f:
            f.write(response.body)
            
# Instantiate our crawler
process = CrawlerProcess()

# Start the crawler with our spider
process.crawl(ESSpider)
process.start()

2019-03-05 20:10:38 [scrapy.utils.log] INFO: Scrapy 1.6.0 started (bot: scrapybot)
2019-03-05 20:10:38 [scrapy.utils.log] INFO: Versions: lxml 4.3.2.0, libxml2 2.9.9, cssselect 1.0.3, parsel 1.5.1, w3lib 1.20.0, Twisted 18.9.0, Python 3.7.0 (default, Aug 17 2018, 21:14:48) - [Clang 9.1.0 (clang-902.0.39.2)], pyOpenSSL 19.0.0 (OpenSSL 1.1.1b  26 Feb 2019), cryptography 2.6.1, Platform Darwin-18.2.0-x86_64-i386-64bit
2019-03-05 20:10:38 [scrapy.crawler] INFO: Overridden settings: {}
2019-03-05 20:10:38 [scrapy.extensions.telnet] INFO: Telnet Password: 0c8285b7bd0c0ac4
2019-03-05 20:10:38 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2019-03-05 20:10:38 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.Downloa

# Parsing HTML
- Using xpath

In [1]:
# importing in each cell because of the kernel restarts
import scrapy
from scrapy.crawler import CrawlerProcess

class ESSpider(scrapy.Spider):
    
    name = 'ESS'
    
    # URL(s) to start with
    start_urls = [
        'http://www.everydaysexism.com'
    ]
    
    # Use XPath to parse the response we get
    def parse(self, response):
        
        # Iterate over every <article> element on the page
        for article in response.xpath('//article'):
            
            # Yield a dictionary with the values we want
            yield {
                # This is the ode to choose what we want to extract
                # Use Xpath expressions if other info wanted
                'name': article.xpath('header/h2/a/@title').extract_first(),
                'date': article.xpath('header/section/span[@class="entry-date"]/text()').extract_first(),
                'text': article.xpath('section[@class="entry-content"]/p/text()').extract(),
                'tags': article.xpath('*/span[@class="tag-links"]/a/text()').extract()
            }

# Tell the script how to run the crawler by passing in settings
process = CrawlerProcess({
    'FEED_FORMAT': 'json',         # Store data in JSON format
    'FEED_URI': 'firstpage2.json',  # Name our storage file
    'LOG_ENABLED': False           # Turn off logging for now
})

# Start the crawler with our spider
process.crawl(ESSpider)
process.start()
print('Success')

Success


# Checking the Output
- Look at firstpage.json

In [2]:
import pandas as pd

firstpage = pd.read_json('firstpage2.json', orient='records')
print(firstpage.shape)
print(firstpage.head())

(10, 4)
        date    name                  tags  \
0 2019-03-02    Fran       [Media, School]   
1 2019-03-02  Charis  [Grandparents house]   
2 2019-03-02    anon                [Home]   
3 2019-03-02    Carl        [Public space]   
4 2019-03-02     aad                    []   

                                                text  
0  [I always believed that have the right to my b...  
1  [I am 12 and was visiting my grandparents over...  
2  [My partner just asked what I was doing, to wh...  
3  [Passed this car while walking with my daughte...  
4  [I was on my way to a concert, my first one ev...  


## Scrapy on more pages

In [2]:
# IMportin in each cell because of kernel restarts
import scrapy
import re
from scrapy.crawler import CrawlerProcess

class ESSpider(scrapy.Spider):
    # Naming spider important if running more than 1
    # of class simultaneously
    name = 'ESS'
    
    # URL(s) to start with
    start_urls = [
        'http://www.everydaysexism.com'
    ]
    
    # Use XPath to parse the response we get.
    def parse(self, response):
        
        # Iterate over every <article> element on the page
        for article in response.xpath('//article'):
            
            # Yield a dictionary with teh values we want
            yield {
                'name': article.xpath('header/h2/a/@title').extract_first(),
                'date': article.xpath('header/section/span[@class="entry-date"]/text()').extract_first(),
                'text': article.xpath('section[@class="entry-content"]/p/text()').extract(),
                'tags': article.xpath('*/span[@class="tag-links"]/a/text()').extract()
            }
        
        # Get the URL of the previous page
        next_page = response.xpath('//div[@class="nav-previous"]/a/@href').extract_first()
        
        # There are a LOT of pages here, we'll just scrape the first 9
        # This finds the page number, next segment prevents going past page 9
        pagenum = int(re.findall(r'\d+', next_page)[0])
        
        # Recursively call the spider to run on the next page if it exists
        if next_page is not None and pagenum < 10:
            next_page = response.urljoin(next_page)
            # Request the next page and recursively parse it the same way we did above
            yield scrapy.Request(next_page, callback=self.parse)
            
# Tell the scrip how to run the crawler by passing in settings
# The new settings have to do with scraping etiquette
process = CrawlerProcess({
    'FEED_FORMAT': 'json',
    'FEED_URI': 'data.json',
    'LOG_ENABLED': False,
    'ROBOTSTXT_OBEY': True,
    'USER_AGENT': 'ThinkfulDataScienceBootcampCrawler (thinkful.com)',
    'AUTOTHROTTLE_ENABLED': True,
    'HTTPCACHE_ENABLED': True
})

# STart the crawler with our spider
process.crawl(ESSpider)
process.start()
print('Success!')
            

        

Success!


In [3]:
import pandas as pd

# Checking whether we got data from all 9 pages
ESSdf = pd.read_json('data.json', orient='records')
print(ESSdf.shape)
print(ESSdf.head())

(90, 4)
        date       name                       tags  \
0 2019-03-06       izzy             [Public space]   
1 2019-03-06       Emma                   [School]   
2 2019-03-06  Normalmum       [Home, Public space]   
3 2019-03-06        Eve             [Public space]   
4 2019-03-06  Jay Again  [Public space, Workplace]   

                                                text  
0  [When I was eleven, I was walking into a petro...  
1  [About a week ago me and my friend were walkin...  
2  [Husband just not understanding why it’s not o...  
3  [Was at TraderJoes shopping this afternoon, an...  
4  [Have you seen this? I’m sure you have. Is the...  
